In [106]:
import torch
from torchvision import datasets, transforms as T

In [107]:
import numpy as np
import pandas as pd
import os
mapa_condiciones = {
    0: "normal",
    1: "tuberculosis",
    2: "neumonia",
    3: "covid-19"
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [26]:
%cd /content/drive/My\ Drive/ProyectoDIALUNG/
#Archivo local 
#%cd /media/josejacomeb/Datos1/DatasetsMédicos/ProyectoDIALUNG/

/media/josejacomeb/Datos1/DatasetsMédicos/ProyectoDIALUNG


In [108]:
from torch.utils.data import ConcatDataset
direccion_dataset_imagenes = "DIALUNG-Datasets/"
direccion_global = os.getcwd()
dir_dataset_train = os.path.join(direccion_global, direccion_dataset_imagenes, "train_256.hdf5") 
dir_dataset_test = os.path.join(direccion_global, direccion_dataset_imagenes, "test_256.hdf5")
dir_dataset_val = os.path.join(direccion_global, direccion_dataset_imagenes, "val_256.hdf5")

In [109]:
#Librerias de DIALUNG
from dialung.utils.creardatasetbinario import crearDatasetBinarioDIALUNG
transform = T.Compose([T.CenterCrop(224)])
center_crop = T.CenterCrop(224)
transform = None
dataset_train = crearDatasetBinarioDIALUNG(dir_dataset_train, transform, rgb = False, correccion_gamma = None)
dataset_test = crearDatasetBinarioDIALUNG(dir_dataset_test, transform, rgb = False, correccion_gamma = None)
dataset_val = crearDatasetBinarioDIALUNG(dir_dataset_val, transform, rgb = False, correccion_gamma = None)

dataset_total = ConcatDataset([dataset_train, dataset_test, dataset_val])

In [110]:
from torch.utils.data import DataLoader #Clase dataloader para cargar en PyTorch los objetos de tipo Dataset
batch_size = 512
#Creo objetos de tipo DataLoader en lotes de 64 imagenes y barajeados aleatoriamente
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers = 2)
test_dataloader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, num_workers = 2)
val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True, num_workers = 2)
total_dataloader = DataLoader(dataset_total, batch_size=batch_size, shuffle=True, num_workers = 2)

In [111]:
import tqdm.notebook as tq
dataloader = total_dataloader
means = []
stds = []
maxs = []
mins = []
total_batches = len(dataloader)
for imgs, labels in tq.tqdm(dataloader, total = total_batches):
    imgs = center_crop(imgs) 
    for img in imgs:
        img = img/255.0 
        means.append(torch.mean(img))
        stds.append(torch.std(img))
        maxs.append(img.max())
        mins.append(img.min())

mean = torch.mean(torch.tensor(means))
std = torch.mean(torch.tensor(stds))
max_value = torch.mean(torch.tensor(maxs))
min_value = torch.mean(torch.tensor(mins))
total_elements = len(means)
print(f"std: {std} mean: {mean} max: {max_value}, min: {min_value} total_samples: {total_elements}")

  0%|          | 0/1 [00:00<?, ?it/s]

std: 0.2167416512966156 mean: 0.5674065947532654 max: 0.9339674115180969, min: 0.03992798551917076 total_samples: 501
